In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import datetime

import requests

In [4]:
url_inflation_db = "https://github.com/matublaq/DataBases/raw/main/Inflation.db"
local_path = "Inflation.db"

response = requests.get(url_inflation_db)

if(response.status_code == 200):
    with open(local_path, 'wb') as file:
        file.write(response.content)
    print(f"Inflation database. Downloaded successfully. Status code {response.status_code}")
else:
    print(f"Inflation database. Download failed. Status code {response.status_code}")

################################################################################################################

url_stockmarket_db = "https://github.com/matublaq/DataBases/raw/main/StockMarket.db"
local_path = "StockMarket.db"

response = requests.get(url_stockmarket_db)

if(response.status_code == 200):
    with open(local_path, 'wb') as file:
        file.write(response.content)
    print(f"Stock market database. Downloaded successfully. Status code {response.status_code}")
else:
    print(f"Stock market database. Download failed. Status code {response.status_code}")

Inflation database. Downloaded successfully. Status code 200
Stock market database. Downloaded successfully. Status code 200


In [5]:
conn1 = sqlite3.connect('Inflation.db')
conn2 = sqlite3.connect('StockMarket.db')
cursor1 = conn1.cursor()
cursor2 = conn2.cursor()


cursor1.execute("SELECT name FROM Countries")
countries = cursor1.fetchall()
countries = [country[0] for country in countries]

cursor2.execute("SELECT name FROM Companies")
companies = cursor2.fetchall()
companies = [company[0] for company in companies]


conn1.close()
conn2.close()

print(countries, "\n", companies)

['Algeria', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Australia', 'Austria', 'Bahamas, The', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bolivia', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Costa Rica', "Côte d'Ivoire", 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt, Arab Rep.', 'El Salvador', 'Equatorial Guinea', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia, The', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Rep.', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kenya', 'Korea, Rep.', 'Kuwait', 'Lao PDR', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'L

<p style='font-size: 35px; text-align: center; color: violet;'>Functions</p>

In [18]:
def country_inflation(code):
    conn = sqlite3.connect('Inflation.db')
    cursor = conn.cursor()

    cursor.execute("SELECT id FROM Countries WHERE code = ?", (code,))
    id_usa = cursor.fetchall()[0]
    cursor.execute("SELECT year, inflation_rate FROM Inflation WHERE country_id = ?", id_usa)
    inflation = cursor.fetchall()

    conn.close()

    return pd.DataFrame(inflation, columns=['year', 'inflation_rate'])

def stock_quote_year_price(ticker):
    with sqlite3.connect('StockMarket.db') as conn:
        cursor = conn.cursor()

        cursor.execute("SELECT id FROM Companies WHERE ticker = ?", (ticker,))
        id_company = cursor.fetchall()[0]

        #Average price in dicember
        cursor.execute('''
                    SELECT strftime('%Y', date) as year, MAX(date) as max_date, close as price, volume
                    FROM Stock_quotes
                    WHERE company_id = ?
                    GROUP BY year
        ''', id_company)
        company_stock_year = cursor.fetchall()
        company_stock_year = pd.DataFrame(company_stock_year, columns=['year', 'max_date', 'price', 'volume'])
        company_stock_year.drop(columns=['max_date'], inplace=True)
        return company_stock_year

def price_inflation_adjusted(ticker, country_code):
    prices = stock_quote_year_price(ticker)
    inflations = country_inflation(country_code)
    prices['growth'] = 0

    today = datetime.datetime.now()
    for year in prices['year']:
        if year == str(today.year):
            return prices
        
        inflation = inflations[inflations['year'] == int(year)]['inflation_rate'].values[0]

        if str(int(year)-1) in prices['year'].values:
            last_price = prices[prices['year'] == str(int(year)-1)]['price'].values[0]
            price_adj_inflation = last_price*((inflation/100) + 1)
            actual_price = prices[prices['year'] == year]['price'].values[0]
            
            diff_adj_actual = ((actual_price - price_adj_inflation)/price_adj_inflation)*100 #Creció o decreció.
            prices.loc[prices['year'] == year, 'growth'] = diff_adj_actual
            real_actual_price = last_price*((diff_adj_actual/100) + 1)
                         
            prices.loc[prices['year'] == year, 'price'] = real_actual_price
        else:
            pass
            
    return prices


In [7]:
inflations = country_inflation('USA')
inflations

,year,inflation_rate
0,1970,5.90
1,1971,4.29
2,1972,3.27
3,1973,6.18
4,1974,11.05
5,1975,9.14
6,1976,5.74
7,1977,6.50
8,1978,7.63
9,1979,11.25


In [8]:
prices = stock_quote_year_price('AAPL')
prices.columns

Index(['year', 'price', 'volume'], dtype='object')

In [9]:
year = '2020'
inflation = inflations[inflations['year'] == year]['inflation_rate']
last_price = prices[prices['year'] == str(int(year)-1)]['price'].values[0]
price_adj_inflation = last_price*((inflation/100) + 1)
actual_price = prices[prices['year'] == year]['price'].values[0]

diff_adj_actual = ((actual_price - price_adj_inflation)/price_adj_inflation)*100 #Creció o decreció.
prices.loc[prices['year'] == year, 'growth'] = diff_adj_actual
real_actual_price = last_price*((diff_adj_actual/100) + 1)
                
prices.loc[prices['year'] == year, 'price'] = real_actual_price



In [15]:
inflations[inflations['year'] == int(year)]['inflation_rate'].values[0]

1.23

In [19]:
prices_inflation = price_inflation_adjusted('AAPL', 'USA')
prices_inflation

C:\Users\matia\AppData\Local\Temp\ipykernel_30948\3268688500.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-40.966573962323544' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  prices.loc[prices['year'] == year, 'growth'] = diff_adj_actual


,year,price,volume,growth
0,1980,0.152000,35750400,0.000000
1,1981,0.089731,54656000,-40.966574
2,1982,0.125318,49660800,39.659956
3,1983,0.105610,91862400,-15.726459
4,1984,0.124640,207760000,18.019650
5,1985,0.094640,87248000,-24.069383
6,1986,0.177625,132563200,87.684505
7,1987,0.361760,117600000,103.664658
8,1988,0.344927,81692800,-4.652985
9,1989,0.300487,152409600,-12.884025
